In [68]:
import numpy as np
import pandas as pd

from docplex.mp.model_reader import ModelReader
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp

In [69]:
dados = pd.read_csv(filepath_or_buffer='dados_batalha_quantum.csv',
                        index_col=0)

In [70]:
dados

,produto,classe,retorno,avaliacao_risco,aloc,ret_med,aval
0,produto_0,classe_0,0.129244,23,0.0,0.0,0.0
1,produto_1,classe_0,0.135872,12,0.0,0.0,0.0
2,produto_2,classe_0,0.118227,5,0.0,0.0,0.0
3,produto_3,classe_0,0.138654,37,0.0,0.0,0.0
4,produto_4,classe_0,0.117790,4,0.0,0.0,0.0
...,...,...,...,...,...,...,...
663,produto_663,classe_9,0.122159,32,0.0,0.0,0.0
664,produto_664,classe_9,0.119627,10,0.0,0.0,0.0
665,produto_665,classe_9,0.197897,45,0.0,0.0,0.0
666,produto_666,classe_9,0.205654,48,0.0,0.0,0.0


In [71]:
covariancia = pd.read_csv(filepath_or_buffer="matriz_covariancia_produtos_quantum.csv",
                              index_col=0)


In [72]:
covariancia

,produto_0,produto_1,produto_2,produto_3,produto_4,produto_5,produto_6,produto_7,produto_8,produto_9,...,produto_658,produto_659,produto_660,produto_661,produto_662,produto_663,produto_664,produto_665,produto_666,produto_667
nome,,,,,,,,,,,,,,,,,,,,,
produto_0,0.005705,0.001248,0.000119,0.001217,0.000027,0.004616,0.000127,-0.004726,-0.001960,-0.002444,...,-0.003688,0.001628,0.000146,0.002059,0.001186,0.000651,0.000054,0.011396,0.011880,0.011821
produto_1,0.001248,0.004568,0.000014,0.001193,0.000090,0.002768,0.000140,-0.001840,-0.000924,-0.000580,...,-0.002154,0.000999,0.000098,0.001285,0.000770,0.000507,0.000009,0.007193,0.007482,0.007470
produto_2,0.000119,0.000014,0.003328,0.000011,0.000007,0.000054,0.000009,-0.000077,-0.000146,-0.000206,...,-0.000145,0.000028,-0.000007,0.000099,0.000020,-0.000020,0.000031,0.000123,0.000191,0.000152
produto_3,0.001217,0.001193,0.000011,0.004467,0.000046,0.002517,-0.000008,-0.001886,-0.000938,-0.000664,...,-0.002392,0.000901,0.000001,0.001259,0.000683,0.000411,0.000051,0.006780,0.006988,0.007058
produto_4,0.000027,0.000090,0.000007,0.000046,0.003326,0.000111,0.000075,0.000025,0.000027,-0.000052,...,-0.000050,0.000086,0.000017,0.000050,0.000021,0.000079,0.000041,0.000232,0.000261,0.000300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
produto_663,0.000651,0.000507,-0.000020,0.000411,0.000079,0.003641,0.000285,-0.000955,0.001750,-0.000017,...,0.001263,0.001360,0.000189,0.001128,0.000951,0.004108,0.000078,0.010386,0.010638,0.010582
produto_664,0.000054,0.000009,0.000031,0.000051,0.000041,0.000350,0.000075,-0.000211,0.000077,0.000044,...,-0.000009,0.000059,0.000044,0.000083,-0.000003,0.000078,0.003332,0.000846,0.000860,0.000858
produto_665,0.011396,0.007193,0.000123,0.006780,0.000232,0.052676,0.003154,-0.027435,0.015722,-0.016545,...,0.000468,0.018939,0.002220,0.017923,0.014257,0.010386,0.000846,0.156487,0.157408,0.157241


In [73]:
sample, _ = prepara_amostra_problema(seed=42)
sample

,produto,classe,ret_med,aval,aloc
49,produto_49,classe_1,0.030658,2,0.3
33,produto_33,classe_1,0.044948,1,0.3
82,produto_82,classe_2,0.036235,0,0.3
75,produto_75,classe_2,0.038913,0,0.3
107,produto_107,classe_3,0.093379,3,0.4
103,produto_103,classe_3,0.035152,2,0.4


In [74]:
def prepara_amostra_problema(seed: int) -> tuple:
    """Função que prepara uma amostra para
    definir um problema de otimização para ser
    resolvido usando computação quântica.

    Args:
        seed (int): Seed.

    Returns:
        tuple: Uma tupla onde o primeiro elemento
        é o DataFrame da amostra e o segundo é a
        matriz de covariância multiplicada pela
        alocação. 
    """

    dados = pd.read_csv(filepath_or_buffer='dados_batalha_quantum.csv',
                        index_col=0)
    covariancia = pd.read_csv(filepath_or_buffer="matriz_covariancia_produtos_quantum.csv",
                              index_col=0)

    np.random.seed(seed)

    cols = ['produto',
            'classe',
            'ret_med',
            'aval',
            'aloc']
    dict_alocacao = {"classe_1": 0.3,
                     "classe_2": 0.3,
                     "classe_3": 0.4}
    
    sample = dados[dados.classe.isin(dict_alocacao.keys())].groupby("classe").sample(2, random_state=seed)
    sample['aval'] = (10*sample.aval/sample.aval.sum()).round(1).astype(int)
    sample = sample[cols].copy()
    
    aloc_matriz = np.outer(sample.aloc.values, sample.aloc.values)
    cov_sample = covariancia.loc[sample.produto.values][sample.produto.values].values*aloc_matriz
    
    return sample, cov_sample


In [76]:
sample

,produto,classe,ret_med,aval,aloc
49,produto_49,classe_1,0.030658,2,0.3
33,produto_33,classe_1,0.044948,1,0.3
82,produto_82,classe_2,0.036235,0,0.3
75,produto_75,classe_2,0.038913,0,0.3
107,produto_107,classe_3,0.093379,3,0.4
103,produto_103,classe_3,0.035152,2,0.4


In [82]:
cov_sample

array([[4.18052618e-03, 6.13401197e-04, 2.98842403e-04, 2.99268725e-04,
        2.33353788e-03, 8.69345850e-04],
       [6.13401197e-04, 4.92017458e-04, 8.41192812e-05, 8.71304619e-05,
        2.23595453e-04, 1.94257115e-04],
       [2.98842403e-04, 8.41192812e-05, 3.67718933e-04, 4.55926108e-05,
        4.88326722e-05, 1.03391737e-04],
       [2.99268725e-04, 8.71304619e-05, 4.55926108e-05, 3.62595691e-04,
        7.38996825e-05, 9.99976407e-05],
       [2.33353788e-03, 2.23595453e-04, 4.88326722e-05, 7.38996825e-05,
        5.12195021e-03, 3.80150882e-04],
       [8.69345850e-04, 1.94257115e-04, 1.03391737e-04, 9.99976407e-05,
        3.80150882e-04, 8.79992133e-04]])

In [112]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import scipy
import seaborn as sns

In [113]:
lista_produtos = pd.read_csv(filepath_or_buffer="dados_batalha_classico.csv", index_col=0)

In [114]:
lista_produtos

,produto,classe,retorno,avaliacao_risco,aloc,ret_med,aval
0,produto_0,classe_0,0.129244,23,0.04,0.005170,0.92
1,produto_1,classe_0,0.135872,12,0.04,0.005435,0.48
2,produto_2,classe_0,0.118227,5,0.04,0.004729,0.20
3,produto_3,classe_0,0.138654,37,0.04,0.005546,1.48
4,produto_4,classe_0,0.117790,4,0.04,0.004712,0.16
...,...,...,...,...,...,...,...
663,produto_663,classe_9,0.122159,32,0.10,0.012216,3.20
664,produto_664,classe_9,0.119627,10,0.10,0.011963,1.00
665,produto_665,classe_9,0.197897,45,0.10,0.019790,4.50
666,produto_666,classe_9,0.205654,48,0.10,0.020565,4.80


In [115]:
matriz_de_covariancia = pd.read_csv(filepath_or_buffer="covariancia_classico.csv", index_col=0)

In [116]:
matriz_de_covariancia

,produto_0,produto_1,produto_2,produto_3,produto_4,produto_5,produto_6,produto_7,produto_8,produto_9,...,produto_658,produto_659,produto_660,produto_661,produto_662,produto_663,produto_664,produto_665,produto_666,produto_667
nome,,,,,,,,,,,,,,,,,,,,,
produto_0,9.127837e-06,1.997092e-06,1.907910e-07,1.946977e-06,4.344784e-08,7.385602e-06,2.024246e-07,-7.561183e-06,-3.135964e-06,-3.910114e-06,...,-1.475392e-05,6.512619e-06,5.842226e-07,8.236342e-06,4.743256e-06,2.604917e-06,2.167275e-07,4.558311e-05,4.752190e-05,4.728425e-05
produto_1,1.997092e-06,7.308937e-06,2.173131e-08,1.908224e-06,1.440145e-07,4.428016e-06,2.247668e-07,-2.943755e-06,-1.478506e-06,-9.272513e-07,...,-8.617950e-06,3.995716e-06,3.939369e-07,5.141105e-06,3.081143e-06,2.029255e-06,3.669501e-08,2.877069e-05,2.992720e-05,2.987897e-05
produto_2,1.907910e-07,2.173131e-08,5.325114e-06,1.722893e-08,1.053738e-08,8.710767e-08,1.370160e-08,-1.224788e-07,-2.337944e-07,-3.298871e-07,...,-5.817326e-07,1.105320e-07,-2.924006e-08,3.959292e-07,7.980783e-08,-7.930616e-08,1.223667e-07,4.933548e-07,7.632671e-07,6.091602e-07
produto_3,1.946977e-06,1.908224e-06,1.722893e-08,7.146463e-06,7.310698e-08,4.027246e-06,-1.201524e-08,-3.018323e-06,-1.500099e-06,-1.061948e-06,...,-9.568140e-06,3.602900e-06,4.085647e-09,5.035034e-06,2.733777e-06,1.644535e-06,2.034310e-07,2.712052e-05,2.795082e-05,2.823313e-05
produto_4,4.344784e-08,1.440145e-07,1.053738e-08,7.310698e-08,5.322298e-06,1.780346e-07,1.200626e-07,3.954907e-08,4.264531e-08,-8.368717e-08,...,-1.993395e-07,3.428925e-07,6.865860e-08,2.016628e-07,8.436638e-08,3.170273e-07,1.648742e-07,9.267664e-07,1.045393e-06,1.200111e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
produto_663,2.604917e-06,2.029255e-06,-7.930616e-08,1.644535e-06,3.170273e-07,1.456430e-05,1.139831e-06,-3.819926e-06,6.999145e-06,-6.691264e-08,...,1.262681e-05,1.359764e-05,1.891298e-06,1.128055e-05,9.509793e-06,4.108333e-05,7.776239e-07,1.038611e-04,1.063846e-04,1.058225e-04
produto_664,2.167275e-07,3.669501e-08,1.223667e-07,2.034310e-07,1.648742e-07,1.401798e-06,3.019568e-07,-8.438810e-07,3.079385e-07,1.751460e-07,...,-9.117189e-08,5.947570e-07,4.378595e-07,8.342752e-07,-2.963031e-08,7.776239e-07,3.331725e-05,8.457980e-06,8.599551e-06,8.575762e-06
produto_665,4.558311e-05,2.877069e-05,4.933548e-07,2.712052e-05,9.267664e-07,2.107042e-04,1.261407e-05,-1.097401e-04,6.288862e-05,-6.618064e-05,...,4.680300e-06,1.893853e-04,2.220461e-05,1.792304e-04,1.425672e-04,1.038611e-04,8.457980e-06,1.564870e-03,1.574078e-03,1.572409e-03


In [117]:
dict_quantidade_produtos = {'classe_0': 5,
                            'classe_1': 2,
                            'classe_2': 3,
                            'classe_3': 1,
                            'classe_4': 1,
                            'classe_5': 4,
                            'classe_6': 2,
                            'classe_7': 2,
                            'classe_8': 1,
                            'classe_9': 1}

c = 50

In [118]:
lista_produtos

,produto,classe,retorno,avaliacao_risco,aloc,ret_med,aval
0,produto_0,classe_0,0.129244,23,0.04,0.005170,0.92
1,produto_1,classe_0,0.135872,12,0.04,0.005435,0.48
2,produto_2,classe_0,0.118227,5,0.04,0.004729,0.20
3,produto_3,classe_0,0.138654,37,0.04,0.005546,1.48
4,produto_4,classe_0,0.117790,4,0.04,0.004712,0.16
...,...,...,...,...,...,...,...
663,produto_663,classe_9,0.122159,32,0.10,0.012216,3.20
664,produto_664,classe_9,0.119627,10,0.10,0.011963,1.00
665,produto_665,classe_9,0.197897,45,0.10,0.019790,4.50
666,produto_666,classe_9,0.205654,48,0.10,0.020565,4.80


In [119]:
dict_lista_produtos_classe = {}
for classe in dict_quantidade_produtos.keys():
    dict_lista_produtos_classe[classe] = lista_produtos.loc[lista_produtos['classe'] == classe]


In [120]:
dict_lista_produtos_classe

{'classe_0':        produto    classe   retorno  avaliacao_risco  aloc   ret_med  aval
 0    produto_0  classe_0  0.129244               23  0.04  0.005170  0.92
 1    produto_1  classe_0  0.135872               12  0.04  0.005435  0.48
 2    produto_2  classe_0  0.118227                5  0.04  0.004729  0.20
 3    produto_3  classe_0  0.138654               37  0.04  0.005546  1.48
 4    produto_4  classe_0  0.117790                4  0.04  0.004712  0.16
 5    produto_5  classe_0  0.180801               36  0.04  0.007232  1.44
 6    produto_6  classe_0  0.136053               35  0.04  0.005442  1.40
 7    produto_7  classe_0  0.112355               34  0.04  0.004494  1.36
 8    produto_8  classe_0  0.094034               46  0.04  0.003761  1.84
 9    produto_9  classe_0  0.145904               37  0.04  0.005836  1.48
 10  produto_10  classe_0  0.176723               34  0.04  0.007069  1.36
 11  produto_11  classe_0  0.050713               63  0.04  0.002029  2.52
 12  produto_

In [121]:
dict_x = {classe: cp.Variable(shape=len(produtos), boolean=True) for classe, produtos in dict_lista_produtos_classe.items()}

In [122]:
dict_x

{'classe_0': Variable((18,), var329, boolean=True),
 'classe_1': Variable((33,), var330, boolean=True),
 'classe_2': Variable((40,), var331, boolean=True),
 'classe_3': Variable((15,), var332, boolean=True),
 'classe_4': Variable((76,), var333, boolean=True),
 'classe_5': Variable((77,), var334, boolean=True),
 'classe_6': Variable((145,), var335, boolean=True),
 'classe_7': Variable((9,), var336, boolean=True),
 'classe_8': Variable((200,), var337, boolean=True),
 'classe_9': Variable((53,), var338, boolean=True)}

In [123]:
objective = cp.Minimize(sum(
    dict_x[classe] @ matriz_de_covariancia.loc[produtos['produto'].values][produtos['produto'].values].values @ dict_x[classe] - 
    produtos['retorno'].values.T @ dict_x[classe] +
    cp.sum(produtos['aval'].values @ dict_x[classe]) - c # Condição: Somatório dos fatores da última coluna <= c
    for classe, produtos in dict_lista_produtos_classe.items()
))

In [124]:
constraints = [cp.sum(dict_x[classe]) == quantidade for classe, quantidade in dict_quantidade_produtos.items()]

constraints += [produtos['aval'].values @ dict_x[classe] <= c for classe, produtos in dict_lista_produtos_classe.items()]  


In [125]:
constraints

[Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Expression(AFFINE, NONNEGATIVE, ())),
 Inequality(Expression(AFFINE, NONNEGATIVE, ())),
 Inequality(Expression(AFFINE, NONNEGATIVE, ())),


In [126]:
dict_lista_produtos_classe

{'classe_0':        produto    classe   retorno  avaliacao_risco  aloc   ret_med  aval
 0    produto_0  classe_0  0.129244               23  0.04  0.005170  0.92
 1    produto_1  classe_0  0.135872               12  0.04  0.005435  0.48
 2    produto_2  classe_0  0.118227                5  0.04  0.004729  0.20
 3    produto_3  classe_0  0.138654               37  0.04  0.005546  1.48
 4    produto_4  classe_0  0.117790                4  0.04  0.004712  0.16
 5    produto_5  classe_0  0.180801               36  0.04  0.007232  1.44
 6    produto_6  classe_0  0.136053               35  0.04  0.005442  1.40
 7    produto_7  classe_0  0.112355               34  0.04  0.004494  1.36
 8    produto_8  classe_0  0.094034               46  0.04  0.003761  1.84
 9    produto_9  classe_0  0.145904               37  0.04  0.005836  1.48
 10  produto_10  classe_0  0.176723               34  0.04  0.007069  1.36
 11  produto_11  classe_0  0.050713               63  0.04  0.002029  2.52
 12  produto_

In [127]:
problem = cp.Problem(objective=objective, constraints=constraints)

In [146]:
problem

Problem(Minimize(Expression(CONVEX, UNKNOWN, ())), [Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Expression(AFFINE, NONNEGATIVE, ())), Inequality(Expression(AFFINE, NONNEGATIVE, ())), Inequality

In [147]:
print(cp.installed_solvers())

['CLARABEL', 'CPLEX', 'ECOS', 'ECOS_BB', 'OSQP', 'SCIPY', 'SCS']


In [148]:
import cvxpy as cp

start_time = time.time()
resultado = problem.solve(solver=cp.ECOS_BB, verbose=False)
end_time = time.time()
execution_time = end_time - start_time


In [149]:
print(f"Tempo de execução: {execution_time:.2f} segundos")

print("Valor da função objetiva (risco total):", resultado)

print("Produtos escolhidos por classe:")

somatorio = 0
for classe, produtos in dict_lista_produtos_classe.items():

    if 'aval' in produtos.columns:
        indices_escolhidos = [i for i in range(len(produtos)) if dict_x[classe].value[i] > 0.5]
        
        produtos_escolhidos = [produtos['aval'].iloc[i] for i in indices_escolhidos]
        produtos_selecionados = [(produtos['produto'].iloc[i]) for i in indices_escolhidos]
#         print(f"Produtos escolhidos {classe}: {produtos_selecionados, produtos_escolhidos}")
                
        print(f"{classe}:")
        display(dict_lista_produtos_classe[classe][dict_x[classe].value.astype(bool)])
        semi_somatorio =0
        for i in indices_escolhidos: 
            semi_somatorio += produtos['aval'].iloc[i]
        somatorio += semi_somatorio
 
    else:
        print(f"Classe {classe}: Coluna 'aval' não encontrada no DataFrame.")
        
print(f"Somatorio aval = {somatorio}")


Tempo de execução: 0.55 segundos
Valor da função objetiva (risco total): -495.5161994642004
Produtos escolhidos por classe:
classe_0:


,produto,classe,retorno,avaliacao_risco,aloc,ret_med,aval
0,produto_0,classe_0,0.129244,23,0.04,0.005170,0.92
1,produto_1,classe_0,0.135872,12,0.04,0.005435,0.48
2,produto_2,classe_0,0.118227,5,0.04,0.004729,0.20
3,produto_3,classe_0,0.138654,37,0.04,0.005546,1.48
4,produto_4,classe_0,0.117790,4,0.04,0.004712,0.16
5,produto_5,classe_0,0.180801,36,0.04,0.007232,1.44
6,produto_6,classe_0,0.136053,35,0.04,0.005442,1.40
7,produto_7,classe_0,0.112355,34,0.04,0.004494,1.36
8,produto_8,classe_0,0.094034,46,0.04,0.003761,1.84
9,produto_9,classe_0,0.145904,37,0.04,0.005836,1.48


classe_1:


,produto,classe,retorno,avaliacao_risco,aloc,ret_med,aval
18,produto_18,classe_1,0.180973,33,0.05,0.009049,1.65
19,produto_19,classe_1,0.180482,45,0.05,0.009024,2.25
20,produto_20,classe_1,0.135729,13,0.05,0.006786,0.65
21,produto_21,classe_1,0.139059,15,0.05,0.006953,0.75
22,produto_22,classe_1,0.131933,25,0.05,0.006597,1.25
23,produto_23,classe_1,0.107326,46,0.05,0.005366,2.30
24,produto_24,classe_1,0.143288,27,0.05,0.007164,1.35
25,produto_25,classe_1,0.185482,44,0.05,0.009274,2.20
26,produto_26,classe_1,0.150200,44,0.05,0.007510,2.20
27,produto_27,classe_1,0.158818,40,0.05,0.007941,2.00


classe_2:


...

classe_8:


,produto,classe,retorno,avaliacao_risco,aloc,ret_med,aval
415,produto_415,classe_8,0.206001,49,0.1,0.020600,4.9
416,produto_416,classe_8,0.072617,40,0.1,0.007262,4.0
417,produto_417,classe_8,0.204549,48,0.1,0.020455,4.8
418,produto_418,classe_8,0.205392,49,0.1,0.020539,4.9
419,produto_419,classe_8,0.253852,74,0.1,0.025385,7.4
...,...,...,...,...,...,...,...
610,produto_610,classe_8,0.228379,62,0.1,0.022838,6.2
611,produto_611,classe_8,0.193368,43,0.1,0.019337,4.3
612,produto_612,classe_8,0.197336,45,0.1,0.019734,4.5
613,produto_613,classe_8,0.117885,2,0.1,0.011789,0.2


classe_9:


,produto,classe,retorno,avaliacao_risco,aloc,ret_med,aval
615,produto_615,classe_9,0.199678,46,0.1,0.019968,4.6
616,produto_616,classe_9,0.234271,65,0.1,0.023427,6.5
617,produto_617,classe_9,0.146016,18,0.1,0.014602,1.8
618,produto_618,classe_9,0.199165,46,0.1,0.019916,4.6
619,produto_619,classe_9,0.130538,8,0.1,0.013054,0.8
620,produto_620,classe_9,0.108792,39,0.1,0.010879,3.9
621,produto_621,classe_9,0.204787,49,0.1,0.020479,4.9
622,produto_622,classe_9,0.137321,12,0.1,0.013732,1.2
623,produto_623,classe_9,0.205497,49,0.1,0.020550,4.9
624,produto_624,classe_9,0.089461,10,0.1,0.008946,1.0


Somatorio aval = 7.217499999999999
